---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [19]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [20]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [21]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    raw_data = pd.read_table('university_towns.txt', names=['Raw data'])

    temp_univ = []
    for i in raw_data.index:
        if raw_data.iloc[i]['Raw data'][-6:] == '[edit]':
            state = raw_data.iloc[i]['Raw data'][:-6]
        elif '(' in raw_data.iloc[i]['Raw data']:
            RegionName = raw_data.iloc[i]['Raw data'][:(raw_data.iloc[i]['Raw data'].index('(')-1)]
            temp_univ.append([state, RegionName])
        else:
            RegionName = raw_data.iloc[i]['Raw data'].rstrip()
            temp_univ.append([state, RegionName])
            
    univ = pd.DataFrame(temp_univ, columns=['State','RegionName'])

    return univ

In [22]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=219, names=['Year_Q','GDP'])
    
    for i in range(2,len(gdp.index)):
        #print(gdp['Year_Q'][i-2],'-', gdp['GDP'][i-2],'  ', 
        #      gdp['Year_Q'][i-1],'-', gdp['GDP'][i-1],'  ', 
        #      gdp['Year_Q'][i],'-', gdp['GDP'][i])
        
        if gdp['GDP'][i-2] > gdp['GDP'][i-1] > gdp['GDP'][i]:
            #print('---',gdp['Year_Q'][i-1],'---')
            return gdp['Year_Q'][i-1]

In [23]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=219, names=['Year_Q','GDP'])
    start = get_recession_start()
    
    for i in range(gdp.index[gdp['Year_Q'] == start][0],len(gdp.index)):
        #print(gdp['Year_Q'][i-2],'-', gdp['GDP'][i-2],'  ', 
        #      gdp['Year_Q'][i-1],'-', gdp['GDP'][i-1],'  ', 
        #      gdp['Year_Q'][i],'-', gdp['GDP'][i])
        
        if gdp['GDP'][i-2] < gdp['GDP'][i-1] < gdp['GDP'][i]:
            #print('---',gdp['Year_Q'][i],'---')
            return gdp['Year_Q'][i]

In [24]:
def get_recession_bottom():
    
    gdp = pd.read_excel('gdplev.xls', usecols=[4,6], skiprows=219, names=['Year_Q','GDP'])
    start = get_recession_start()
    end = get_recession_end()
    
    recession = gdp.loc[gdp.index[gdp['Year_Q'] == start][0]:
                        gdp.index[gdp['Year_Q'] == end][0]].set_index('Year_Q')

    return recession['GDP'].idxmin()

In [25]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #create and clean data
    housing_raw = pd.read_csv('City_Zhvi_AllHomes.csv')
    hm = housing_raw.drop(housing_raw.columns[3:51],axis=1).drop(housing_raw.columns[0],axis=1)
    hm['State'] = hm['State'].map(states)
    hm = hm.set_index(['State', 'RegionName'])
    
    #new columns in required format
    y = list(map(str, range(2000,2017)))
    q = ['q1','q2','q3','q4']
    new_columns = []
    for i in y:
        for j in q:
            new_columns.append(i+j)
    new_columns = new_columns[:67]
    
    #calculating values for new columns
    j=0
    for i in range(0, len(hm.columns), 3):
        hm[new_columns[j]] = hm[hm.columns[i:i+3]].mean(axis=1)
        j+=1

    hm = hm[new_columns]

    return hm

In [27]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    univ_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    housing_data = convert_housing_data_to_quarters()
    
    housing_univ = pd.merge(univ_towns, housing_data, how='inner', left_index=True, right_index=True).dropna()
    housing_non_univ = pd.merge(univ_towns, housing_data, how='outer', left_index=True, right_index=True).dropna()
    
    #get quarter before recession
    year = int(get_recession_start()[:4])
    q = int(get_recession_start()[-1:])
    
    if q in range(2,4):
        q_before_recession = str(year)+'q'+str(q-1)
    if q == 1:
        q_before_recession = str(year-1)+'q'+str(q+3)

    #another option - better :)
    #q_before_recession = housing_data.columns[housing_data.columns.get_loc(get_recession_start()) - 1]
        
        
    #price ratio
    price_ratio_univ = housing_univ[q_before_recession] / housing_univ[get_recession_bottom()]
    price_ratio_non_univ = housing_non_univ[q_before_recession] / housing_non_univ[get_recession_bottom()]
    
    #get p from ttest
    p = ttest_ind(price_ratio_univ, price_ratio_non_univ)[1]
    
    #set different
    if p < 0.01:
        different = True
    if p >= 0.01:
        different = False
        
    #set better
    if price_ratio_univ.mean() < price_ratio_non_univ.mean():
        better = 'university town'
    if price_ratio_univ.mean() > price_ratio_non_univ.mean():
        better = 'non-university town'
    

    return (different, p, better)
run_ttest()

(True, 0.0062357516955056733, 'university town')